# Mobile App for Lottery Addiction

In this guided project, we will contribute to the development of a mobile app that is meant to help lottery addicts better estimate their chances of winning.

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49](https://en.wikipedia.org/wiki/Lotto_6/49) lottery and build functions that enable users to answer questions like:

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. [The data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018 (we'll come back to this).

**Disclaimer:** The project is prepared as part of the guided projects on dataquest where I had an active subscription. While most of the text is written based on the guided project content, the code and the analysis is my work.

## Core Functions

Throughout the project, we'll need to calculate repeatedly probabilities and combinations. As a consequence, we'll start by writing two functions that we'll use often:

* A function that calculates factorials; and
* A function that calculates combinations.

In [7]:
# function to calculate factorials
def factorial(n):
    return n * factorial(n-1) if n > 1 else 1

# function to calculate combinations
def combinations(n, k):
    numerator = factorial(n)
    denominator = factorial(n-k) * factorial(k)
    return numerator / denominator

## One-ticket Probability

In the previous section, we wrote two functions which we will use often. Next, we will write a function that *calculates the probability of winning the big prize*.

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
* The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

Let's write this function!

In [89]:
# function discussed above
def one_ticket_probability(list_of_six):
    # calculate possible outcomes
    possible_outcomes = combinations(49, int(len(list_of_six)))
    
    # single outcome
    single_outcome = 1
    
    # calculate probability
    prob = single_outcome / possible_outcomes
    
    return "Probability of this ticket {} winning is {:.6%}. \
            In other words, your chances are 1 in {:,}".format(list_of_six, prob, possible_outcomes)

In [90]:
one_ticket_probability([4, 5, 6, 8, 34, 25])

'Probability of this ticket [4, 5, 6, 8, 34, 25] winning is 0.000007%.             In other words, your chances are 1 in 13,983,816.0'

In the above function, we take a list of six integers and calculate the probability that this exact combination will come out in the lottery. The function takes in a list and uses the combinations function to find the total number of possible outcomes for a six integer list and then divides one by that amount. One, because this is an exact order in which the ticket should come out in order to win.

## Historical Data Check for Canada Lottery

Above, we wrote a function that can tell users what is the probability of winning the big prize with a single ticket. For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

Thus, we'll focus on exploring the historical data coming from the Canada 6/49 lottery. The data set can be downloaded from [Kaggle](https://www.kaggle.com/datascienceai/lottery-dataset) and it has the following structure:

In [51]:
import pandas as pd

# load the dataset
data = pd.read_csv('649.csv')

# check top records
data.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [50]:
# check bottom records
data.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, we can find the six numbers drawn in the following six columns:

* `NUMBER DRAWN 1` - .. - `NUMBER DRAWN 6`

## Function for Historical Data Check

Now that we are familiar with our dataset, we are going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team told us that we need to be aware of the following details:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list and serve as an input to our function.
* The engineering team wants us to write a function that prints:
    * the number of times the combination selected occurred in the Canada data set; and
    * the probability of winning the big prize in the next drawing with that combination.
    
We'll now write this function.

In [61]:
# function to extract the six numbers from the dataset
def extract_numbers(row):
    
    # get the necessary number columns
    cols = row[row.index.str.contains("NUMBER DRAWN")]
    
    # return the set
    return {s for s in cols}

In [74]:
# extract the number sets
number_sets = data.apply(extract_numbers, axis=1)
number_sets.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

The function above takes each row of the dataframe, filters the necessary columns that contain the number drawn and then iterates through each value to create a set. The dataframe.apply simply does this for each row in the dataframe, generating sets for each record.

In [72]:
# function to check historical ocurrences
def check_historical_occurence(user_numbers, number_sets):
    
    # convert user number list to set
    user_numbers = set(user_numbers)
    
    # compare the user number set to the extracted number sets
    # returns boolean series
    comparison = number_sets == user_numbers
    
    # count the number of times the combination matched
    match = comparison.sum()
    
    # print the result of the match count
    print("This combination ocurred {} times in the past".format(match))
    
    # calculate the chance of winning with this ticket
    print(one_ticket_probability(user_numbers))

In [91]:
check_historical_occurence([4, 9, 42, 44, 15, 20], number_sets)

This combination ocurred 1 times in the past
Probability of this ticket {4, 9, 42, 44, 15, 20} winning is 0.000007%.             In other words, your chances are 1 in 13,983,816.0


The function above, simply calculates the number of times a certain combination of numbers in a ticket has ocurred in the past and then using the function we wrote previously to calculate the probability of getting the occurence informs the user about his/her chances of winning. It's important to note that historical performance of a set doesn't define in any way its chances of ocurrence.

## Multi-ticket probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

* The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
* Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
* The function should print information about the probability of winning the big prize depending on the number of different tickets played.

Let's write this function.

In [116]:
# function to calculate multi ticket probability
def multi_ticket_probability(num_tickets):
    
    # total possible outcomes
    possible_outcomes = combinations(49, 6)
    
    # number of successfull outcomes is the num_tickets
    # calculate the probability of these tickets to win
    probs = num_tickets / possible_outcomes
    
    # print results in a friendly format
    print("If you play {} tickets your chances of winning are {} in {:,}. \n\
    In other words {:.5%} chance".format(num_tickets, num_tickets, possible_outcomes, probs))

The function above takes the number of tickets and calculates the probability of winning for the user possessing this many tickets. Let's test our function using multiple inputs

In [120]:
multi_ticket_probability(1)

If you play 1 tickets your chances of winning are 1 in 13,983,816.0. 
    In other words 0.00001% chance


In [121]:
multi_ticket_probability(1000000)

If you play 1000000 tickets your chances of winning are 1000000 in 13,983,816.0. 
    In other words 7.15112% chance


In [122]:
multi_ticket_probability(6991908)

If you play 6991908 tickets your chances of winning are 6991908 in 13,983,816.0. 
    In other words 50.00000% chance


## Less winning numbers

So far we wrote these functions:

* `one_ticket_probability()` — calculates the probability of winning the big prize with a single ticket
* `check_historical_occurrence()` — checks whether a certain combination has occurred in the Canada lottery data set
* `multi_ticket_probability()` — calculates the probability for any number of tickets between 1 and 13,983,816

Lastly, we're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

* Inside the app, the user inputs:
    * six different numbers from 1 to 49; and
    * an integer between 2 and 5 that represents the number of winning numbers expected
* Our function prints information about the probability of having the inputted number of winning numbers.

In [148]:
# function to calculate the probability of less than 6 numbers, which takes the number less than 6
# indicating the integer of winning numbers expected
def probability_less_6(user_choice):
    
    # calculate combinations given the user choice (6 choose k-user_choice)
    success_outcomes = combinations(6, user_choice)
    
    # calculate possible outcomes given the user choice
    possible_outcomes = combinations(49-user_choice, 6-user_choice)
    
    # calculate total outcomes
    total_outcomes = success_outcomes * possible_outcomes
    
    # calculate total possible outcomes 49 choose 6 combinations
    total_combinations = combinations(49, 6)
    
    # calculate probability of winning 
    prob = total_outcomes / total_combinations
    
    print("Your chance of winning with {} numbers out of 6 is {:.7%}".format(user_choice, prob))

In [149]:
probability_less_6(2)

Your chance of winning with 2 numbers out of 6 is 19.1326531%


In [151]:
probability_less_6(4)

Your chance of winning with 4 numbers out of 6 is 0.1061942%


## Summary

In this guided project, we managed to write four main functions for our app.

* `one_ticket_probability()` — calculates the probability of winning the big prize with a single ticket
* `check_historical_occurrence()` — checks whether a certain combination has occurred in the Canada lottery data set
* `multi_ticket_probability()` — calculates the probability for any number of of tickets between 1 and 13,983,816
* `probability_less_6()` — calculates the probability of having two, three, four or five winning numbers


Possible features for a second version of the app include:

* Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery").
* Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time.